In [2]:
import django
import sys, os

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

import pandas as pd
import numpy as np
import datetime

In [3]:
columns = [
    'id',
    'UT_id',
    'authors',
    'content',
    'first_author',
    'title',
    'wosarticle__de',
    'wosarticle__wc'
]

In [7]:
q = Query.objects.get(pk=1457)

doubletag = Tag.objects.get(pk=1405)

rated_docs = DocOwnership.objects.filter(
    tag__query=1457,relevant__gt=0,tag=doubletag
).values_list('doc__id',flat=True)

ratings = list(Doc.objects.filter(id__in=rated_docs,content__iregex='\w').values(*columns))
for r in ratings:
    d = Doc.objects.get(pk=r['id'])
    cutoff = datetime.datetime(2018,6,1,0,0,0)
    dos = DocOwnership.objects.filter(
        query=q,doc=d,relevant__gt=0,
        date__isnull=False,date__gt=cutoff,tag=doubletag
    ).order_by('date')
    
    if not dos.exists():
        continue
    r['ratings'] = dos.count()
    r['first_rated'] = dos.first().date
    for i, do in enumerate(dos):
        r[f'rating_{i+1}'] = do.relevant
    continue
    
        
df = pd.DataFrame.from_dict(ratings)

df = df[df['rating_1'].notna()]

#df = df[df['relevant'].isin([1,2])]

#df['relevant'][df['relevant']==2] = 0

df = df.reset_index(drop=True)

df.head()


/home/galm/software/django/tmvenv_36/lib/python3.6/site-packages/django/db/models/fields/__init__.py:1421: RuntimeWarning: DateTimeField DocOwnership.date received a naive datetime (2018-06-01 00:00:00) while time zone support is active.
  RuntimeWarning)


,UT_id,authors,content,first_author,first_rated,id,rating_1,ratings,title,wosarticle__de,wosarticle__wc
0,WOS:A1993LQ17900012,"LI, SY, LI, SY",Triple cereal cropping is an important grain p...,"LI, SY",2018-12-17 13:57:43.275821+00:00,50914,3,1,YIELD STABILITY AND FERTILIZER EFFICIENCY OF L...,TRIPLE CEREAL CROPPING; BARLEY; DOUBLE-CROPPED...,[Soil Science]
1,WOS:000265440700004,"Garg, A, Dhiman, RC, Bhattacharya, S, Shukla, PR",India has reasons to be concerned about climat...,"Garg, A",2018-12-17 13:57:46.193178+00:00,715432,1,1,"Development, Malaria and Adaptation to Climate...",Impact assessment; Development; Climate change...,[Environmental Sciences]
2,WOS:000293236800005,"Bencze, L, Carter, L",It is apparent that many of us live in a hyper...,"Bencze, L",2018-12-17 13:57:45.306303+00:00,497013,1,1,Globalizing Students Acting for the Common Good,globalization; science; education; neoconserva...,[Education & Educational Research]
3,WOS:000229230800003,"Cairns, J","How ironic that, in humankind's quest for sust...","Cairns, J",2018-12-17 13:57:46.180195+00:00,705124,1,1,Biomonitoring: The crucial link between natura...,ecosystems; biomonitoring; sustainability of l...,[Anthropology]
4,WOS:A1995RJ43000001,"SPEHAR, CR",The cerrados of Brazil are typical of low-lati...,"SPEHAR, CR",2018-12-17 13:57:43.396902+00:00,71307,1,1,IMPACT OF STRATEGIC GENES IN SOYBEAN ON AGRICU...,BREEDING; GLYCINE MAX; MATURITY; SOIL TOLERANCE,[Agronomy]


In [5]:
df.shape

(200, 11)